# DiD and spillover models
Propensity scoring, DiD, and spillover diagnostics.


In [1]:
import sys
from pathlib import Path

ROOT = (Path(__file__).resolve().parents[1]
        if "__file__" in globals() else Path.cwd().parent)

if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

In [2]:
import pandas as pd
from src.data_wrangling import build_processed_dataset
from src.matching import attach_propensity_scores

gdf = build_processed_dataset()
gdf, ps_model = attach_propensity_scores(gdf)
print(ps_model.summary())
gdf[['propensity_score', 'iptw']].describe()


c:\Users\amahai00\AIRS\eep\src\data_wrangling.py:36: UserWarning: Geometry column does not contain geometry.
  out_df["geometry"] = out_df.geometry.to_wkt()


                           Logit Regression Results                           
Dep. Variable:                treated   No. Observations:                24398
Model:                          Logit   Df Residuals:                    24393
Method:                           MLE   Df Model:                            4
Date:                Fri, 21 Nov 2025   Pseudo R-squ.:                  0.2251
Time:                        18:37:47   Log-Likelihood:                -168.57
converged:                       True   LL-Null:                       -217.55
Covariance Type:            nonrobust   LLR p-value:                 2.678e-20
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                11.4168      5.931      1.925      0.054      -0.208      23.041
Elevation_m          -0.0016      0.001     -1.432      0.152      -0.004       0.001
Rainfall_mm          -0.

,propensity_score,iptw
count,2.448800e+04,24488.000000
mean,1.313541e-03,1.601577
std,1.520974e-02,12.134032
min,3.817287e-07,1.000000
25%,7.664635e-05,1.000077
50%,3.463956e-04,1.000347
75%,8.669539e-04,1.000868
max,8.518460e-01,708.744994


In [3]:
from src.models import run_did, run_spillover

targets = ['EVI', 'NDVI']
did_models = {t: run_did(gdf, t, weight_col='iptw', use_baseline=True) for t in targets}
spill_models = {t: run_spillover(gdf, t) for t in targets}

for name, model in did_models.items():
    print(f'DiD {name} coef (treated:post):', model.params.get('treated:post'))

for name, model in spill_models.items():
    print(f'Spillover {name} coef (adjacent_group:post):', model.params.get('adjacent_group:post'))


DiD EVI coef (treated:post): None
DiD NDVI coef (treated:post): None
Spillover EVI coef (adjacent_group:post): None
Spillover NDVI coef (adjacent_group:post): None


In [4]:
import pandas as pd
effects = []
for t, model in did_models.items():
    effects.append({'outcome': t, 'estimator': 'DiD', 'term': 'treated:post', 'coef': model.params.get('treated:post'), 'pval': model.pvalues.get('treated:post')})
for t, model in spill_models.items():
    effects.append({'outcome': t, 'estimator': 'Spillover', 'term': 'adjacent_group:post', 'coef': model.params.get('adjacent_group:post'), 'pval': model.pvalues.get('adjacent_group:post')})
pd.DataFrame(effects)


,outcome,estimator,term,coef,pval
0,EVI,DiD,treated:post,None,None
1,NDVI,DiD,treated:post,None,None
2,EVI,Spillover,adjacent_group:post,None,None
3,NDVI,Spillover,adjacent_group:post,None,None
